## Аугментация данных при помощи дообученной модели Whisper

#### Настройка CUDA

In [2]:
# export CUDA_VISIBLE_DEVICES=0
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [1]:
import torch

# torch is available

torch.cuda.is_available()

True

#### Инициализация модели с huggingface
Ссылка на модель: [https://huggingface.co/mitchelldehaven/whisper-medium-ru](https://huggingface.co/mitchelldehaven/whisper-medium-ru)

In [1]:
from transformers import pipeline
# whisper pipeline on cuda

whisper  = pipeline("automatic-speech-recognition",
                    "mitchelldehaven/whisper-medium-ru", device=0, batch_size=4)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Парсинг скачанных файлов датасета
Ссылка на датасет: [https://github.com/snakers4/open_stt](https://github.com/snakers4/open_stt)

In [2]:
## dataset from folder audios and text
import os

audio_files = []
for root, dirs, files in os.walk('data/asr_public_phone_calls_1'):
    for file in files:
        if file.endswith('.opus'):
            audio_files.append(os.path.join(root, file))

#### Выделение 15000 образцов

In [3]:
# dataframe from audio files
import pandas as pd

df = pd.DataFrame(audio_files[:15000], columns=['audio_path'])
df['text_path'] = df['audio_path'].str.replace('.opus', '.txt')
# model transcriptions from text files
df['model_annotation'] = df['text_path'].apply(lambda x: open(x, 'r', encoding='utf-8').read())
df.head()

,audio_path,text_path,model_annotation
0,data/asr_public_phone_calls_1\0\00\09b83c7eb91...,data/asr_public_phone_calls_1\0\00\09b83c7eb91...,это вопрос это вспомнить надо\n
1,data/asr_public_phone_calls_1\0\00\0b8a7145390...,data/asr_public_phone_calls_1\0\00\0b8a7145390...,я кстати хочу вас познакомить с барием\n
2,data/asr_public_phone_calls_1\0\00\0f9697a8578...,data/asr_public_phone_calls_1\0\00\0f9697a8578...,кафе шишка слышу уже знать звонила\n
3,data/asr_public_phone_calls_1\0\00\154f8d23228...,data/asr_public_phone_calls_1\0\00\154f8d23228...,я ничего не пойму что вы от меня хотите\n
4,data/asr_public_phone_calls_1\0\00\15cf50b9f7f...,data/asr_public_phone_calls_1\0\00\15cf50b9f7f...,для закрытия\n


In [4]:
df.shape

(15000, 3)

### Транскрипция речи при помощи модели

In [5]:
from tqdm import tqdm

tqdm.pandas()
df['whisper_transcription'] = df['audio_path'].progress_apply(lambda x: whisper(x))
df['whisper_transcription'] = df['whisper_transcription'].apply(lambda x: x['text'])
df['model_annotation'] = df['model_annotation'].apply(lambda x: x.replace('\n', ''))

  0%|          | 0/15000 [00:00<?, ?it/s]c:\Users\Davron\code\.venv\Lib\site-packages\transformers\models\whisper\modeling_whisper.py:697: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  0%|          | 11/15000 [00:07<2:34:19,  1.62it/s]--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Davron\AppData\Local\Programs\Python\Python311\Lib\logging\__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Davron\AppData\Local\Programs\Python\Python311\Lib\logging\__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Davron\AppData\Local\Programs\Python\Python311\Lib\logging\__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "C

In [6]:
df.head()

,audio_path,text_path,model_annotation,whisper_transcription
0,data/asr_public_phone_calls_1\0\00\09b83c7eb91...,data/asr_public_phone_calls_1\0\00\09b83c7eb91...,это вопрос это вспомнить надо,как то бас рол ты тоже помнишь да
1,data/asr_public_phone_calls_1\0\00\0b8a7145390...,data/asr_public_phone_calls_1\0\00\0b8a7145390...,я кстати хочу вас познакомить с барием,я кстати хочу вас познакомить с борей
2,data/asr_public_phone_calls_1\0\00\0f9697a8578...,data/asr_public_phone_calls_1\0\00\0f9697a8578...,кафе шишка слышу уже знать звонила,квадрат и слышать да слышу уже мать звонила
3,data/asr_public_phone_calls_1\0\00\154f8d23228...,data/asr_public_phone_calls_1\0\00\154f8d23228...,я ничего не пойму что вы от меня хотите,я ничего не пойму что вы от меня хотите
4,data/asr_public_phone_calls_1\0\00\15cf50b9f7f...,data/asr_public_phone_calls_1\0\00\15cf50b9f7f...,для закрытия,для закрытия


#### Рассчёт WER между транскрипцией модели и исходной разметки (неизвестной моделью)

In [7]:
import jiwer

df['wer_whisper_model'] = df.apply(lambda x: jiwer.wer(x['whisper_transcription'], x['model_annotation']), axis=1)

#### Разметка Label, основываясь на WER

In [8]:
df['label'] = df['wer_whisper_model'].apply(lambda x: 1 if x > 0.48 else 0)

In [9]:
df.head()

,audio_path,text_path,model_annotation,whisper_transcription,wer_whisper_model,label
0,data/asr_public_phone_calls_1\0\00\09b83c7eb91...,data/asr_public_phone_calls_1\0\00\09b83c7eb91...,это вопрос это вспомнить надо,как то бас рол ты тоже помнишь да,1.000000,1
1,data/asr_public_phone_calls_1\0\00\0b8a7145390...,data/asr_public_phone_calls_1\0\00\0b8a7145390...,я кстати хочу вас познакомить с барием,я кстати хочу вас познакомить с борей,0.142857,0
2,data/asr_public_phone_calls_1\0\00\0f9697a8578...,data/asr_public_phone_calls_1\0\00\0f9697a8578...,кафе шишка слышу уже знать звонила,квадрат и слышать да слышу уже мать звонила,0.625000,1
3,data/asr_public_phone_calls_1\0\00\154f8d23228...,data/asr_public_phone_calls_1\0\00\154f8d23228...,я ничего не пойму что вы от меня хотите,я ничего не пойму что вы от меня хотите,0.000000,0
4,data/asr_public_phone_calls_1\0\00\15cf50b9f7f...,data/asr_public_phone_calls_1\0\00\15cf50b9f7f...,для закрытия,для закрытия,0.000000,0


#### Сохранение Датасета

In [10]:
# save to xlsx
df.to_excel('data/aug_dataset_15000.xlsx', index=False)

#### Чтение датасета

In [11]:
import pandas as pd

df = pd.read_excel('data/aug_dataset_15000.xlsx')
df.head()

,audio_path,text_path,model_annotation,whisper_transcription,wer_whisper_model,label
0,data/asr_public_phone_calls_1\0\00\09b83c7eb91...,data/asr_public_phone_calls_1\0\00\09b83c7eb91...,это вопрос это вспомнить надо,как то бас рол ты тоже помнишь да,1.000000,1
1,data/asr_public_phone_calls_1\0\00\0b8a7145390...,data/asr_public_phone_calls_1\0\00\0b8a7145390...,я кстати хочу вас познакомить с барием,я кстати хочу вас познакомить с борей,0.142857,0
2,data/asr_public_phone_calls_1\0\00\0f9697a8578...,data/asr_public_phone_calls_1\0\00\0f9697a8578...,кафе шишка слышу уже знать звонила,квадрат и слышать да слышу уже мать звонила,0.625000,1
3,data/asr_public_phone_calls_1\0\00\154f8d23228...,data/asr_public_phone_calls_1\0\00\154f8d23228...,я ничего не пойму что вы от меня хотите,я ничего не пойму что вы от меня хотите,0.000000,0
4,data/asr_public_phone_calls_1\0\00\15cf50b9f7f...,data/asr_public_phone_calls_1\0\00\15cf50b9f7f...,для закрытия,для закрытия,0.000000,0


In [12]:
del_col_df = df.loc[:, ['model_annotation', 'label']]

In [13]:
del_col_df.to_csv('data/aug_dataset_15000.csv', index=False)

### Проверка точности транскрипции новой моделью по сравнению с ручной разметкой

#### Чтение исходного датасета с ручной разметкой

In [55]:
# read old dataset from xslx
import pandas as pd

df = pd.read_excel('data/hackaton_result_dataset.xlsx')
# get only 10 rows
# df = df.head(1000)
df.head()

,model_annotation,human_markup,audio_path,label
0,давай по россии значит на коленях быстро блять...,давай проси значит на коленях быстро блять,s3://ap-training-set/xacaton_openstt/part_1/05...,1
1,ну разве можно так с телефоном поступает,ну что ну разве можно так с телефоном поступать,s3://ap-training-set/xacaton_openstt/part_1/05...,0
2,у меня нет с собой в полном адресе я щас дома ...,у меня нет с собой полного адреса я щас из дом...,s3://ap-training-set/xacaton_openstt/part_1/05...,0
3,а я здесь кто я санитар,а я знаешь кто я санитар,s3://ap-training-set/xacaton_openstt/part_1/05...,0
4,дежурный по кузьминскому военнокомату,дежурный по кузьминскому военкомату,s3://ap-training-set/xacaton_openstt/part_1/05...,0


### Вычисление WER между ручной разметкой и транскрипцией неизвестной моделью

In [27]:
# count WER with jiwer

import jiwer

df['wer_human_model'] = df.apply(lambda x: jiwer.wer(x['human_markup'], x['model_annotation']), axis=1)
df.head()

,model_annotation,human_markup,audio_path,label,wer_human_model
0,давай по россии значит на коленях быстро блять...,давай проси значит на коленях быстро блять,s3://ap-training-set/xacaton_openstt/part_1/05...,1,0.428571
1,ну разве можно так с телефоном поступает,ну что ну разве можно так с телефоном поступать,s3://ap-training-set/xacaton_openstt/part_1/05...,0,0.333333
2,у меня нет с собой в полном адресе я щас дома ...,у меня нет с собой полного адреса я щас из дом...,s3://ap-training-set/xacaton_openstt/part_1/05...,0,0.323529
3,а я здесь кто я санитар,а я знаешь кто я санитар,s3://ap-training-set/xacaton_openstt/part_1/05...,0,0.166667
4,дежурный по кузьминскому военнокомату,дежурный по кузьминскому военкомату,s3://ap-training-set/xacaton_openstt/part_1/05...,0,0.250000


In [28]:
# change audio_path to data/filter_audio from s3://ap-training-set/xacaton_openstt/part_1/
df['audio_path'] = df['audio_path'].str.replace('s3://ap-training-set/xacaton_openstt/part_1/', 'data/filter_audio/')
df['audio_path'] = df['audio_path'].str.replace('s3://ap-training-set/xacaton_openstt/part_2/', 'data/filter_audio/')
df['audio_path'] = df['audio_path'].str.replace('s3://ap-training-set/xacaton_openstt/part_3/', 'data/filter_audio/')
df.head()

,model_annotation,human_markup,audio_path,label,wer_human_model
0,давай по россии значит на коленях быстро блять...,давай проси значит на коленях быстро блять,data/filter_audio/05a8baf22e2a.wav,1,0.428571
1,ну разве можно так с телефоном поступает,ну что ну разве можно так с телефоном поступать,data/filter_audio/05a5a51a1e9a.wav,0,0.333333
2,у меня нет с собой в полном адресе я щас дома ...,у меня нет с собой полного адреса я щас из дом...,data/filter_audio/05a4c4f90c59.wav,0,0.323529
3,а я здесь кто я санитар,а я знаешь кто я санитар,data/filter_audio/059d08276851.wav,0,0.166667
4,дежурный по кузьминскому военнокомату,дежурный по кузьминскому военкомату,data/filter_audio/059af3258ca9.wav,0,0.250000


In [29]:
for i in df['audio_path'].tolist():
    if not os.path.exists(i):
        print(i)

#### Транскрипция новой моделью (Whisper)

In [30]:
# transcriptions from whisper

df['whisper_transcription'] = df['audio_path'].progress_apply(lambda x: whisper(x, chunk_length_s=30, batch_size=4))
df['whisper_transcription'] = df['whisper_transcription'].apply(lambda x: x['text'])

100%|██████████| 6508/6508 [1:14:49<00:00,  1.45it/s]


#### Вычисление WER между транскрипцией новой моделью и исходной транскрипцией неизвестной моделью

In [41]:
# add wer_whisper
df['wer_whisper_model'] = df.apply(lambda x: jiwer.wer(x['whisper_transcription'], x['model_annotation']), axis=1)

# label with wer threshold 0.43
df['label_by_whisper'] = df['wer_whisper_model'].apply(lambda x: 1 if x > 0.48 else 0)

# accuracy between label and label_by_whisper
df['accuracy_between_labels'] = df.apply(lambda x: 1 if x['label'] == x['label_by_whisper'] else 0, axis=1)
# sum divide to count accuracy
df['accuracy_between_labels'].sum() / len(df)

0.7997848801475107

### Вывод:
Разметка новой моделью дает 80% точности ручной разметки, что дает надежду на использование данной разметки для аугментации датасета и дальнейшего использования увеличенного датасета для обучения сложных моделей (на основе нейронных сетей)

### Сохранение датасета

In [42]:
df.to_excel('data/dataset_add_whisper.xlsx', index=False)